# Policy Evaluation
## Rea Gianluca 278722

### Imports

In [10]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%pylab inline
import random
import pandas as pd
pd.set_option('display.expand_frame_repr', False)


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/gianlucarea/Desktop/DSWorkspace/env/lib/python3.8/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# First-visit Monte Carlo

In [11]:
### Definitions gridsize, terminationStates, actions_dictionary, actions

In [12]:
gridsize = [16, 9]

terminationStates = [[4,0], [4,7], [12,0], [12,7]]

actions_dictionary = {
    "U":    [-1, 0],
    "D":    [1, 0],
    "R":    [0, 1],
    "L":    [0, -1],
    "W_R":  [0,0],
    "W_L":  [0,0],
    "W_D":  [0,0],
    "W_U":  [0,0],
    "P":    [4,0],
    "T_R":  [0,2],
    "T_L":  [0,-2]
}
#up, down, right, left, pick, travel (to right), travel (to left)

actions_ = {
    #(0,0):('D', 'R', 'L', 'U'),
    ### For eggs in pan
    (0,0): {'R': -1, 'W_L':-1, 'W_D':-1, 'W_U':-1},
    (0,1): {'R': -1, 'L':-1, 'D':-1, 'W_U':-1},
    (0,2): {'R': -1, 'L': -1, 'D': -1, 'W_U':-1 },
    (0,3): {'W_R': -1, 'L': -1, 'D': -1, 'W_U':-1 },
    (0,5): {'R': -1, 'W_L': -1, 'D': -1, 'W_U': -1},
    (0,6): {'R': -1, 'L': -1, 'D': -1, 'W_U': -1},
    (0,7): {'R': -1, 'L': -1, 'W_D': -1, 'W_U': -1},
    (0,8): {'W_R': -1, 'L': -1, 'W_D': -1, 'W_U': -1},

    (1,0): {'W_R': -1, 'W_L':-1, 'D': -1, 'W_U':-1, 'P':-1},
    (1,1): {'R': -1, 'W_L':-1, 'W_D': -1, 'U': -1},
    (1,2): {'R': -1, 'L': -1, 'W_D': -1, 'U': -1 },
    (1,3): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },
    (1,5): {'R': -1, 'W_L': -1, 'D': -1, 'U': -1},
    (1,6): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1},
    (1,7): {'R': -1, 'W_L': -1, 'D': -1, 'W_U': -1, 'P': -1},
    (1,8): {'W_R': -1, 'L': -1, 'D': -1, 'W_U': -1},

    (2,0): {'R': -1, 'W_L':-1, 'W_D': -1, 'U': -1},
    (2,1): {'R': -1, 'L': -1, 'W_D': -1, 'W_U':-1 },
    (2,2): {'R': -1, 'L':-1, 'D': -1, 'W_U': -1 },
    (2,3): {'W_R': -1, 'L':-1, 'D': -1, 'U': -1 },
    (2,5): {'R': -1, 'W_L': -1, 'D': -1, 'U': -1},
    (2,6): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1},
    (2,7): {'R': -1, 'W_L': -1, 'W_D': -1, 'U': -1},
    (2,8): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1},

    (3,0): {'R': -1, 'W_L':-1, 'W_D':-1, 'W_U':-1 },
    (3,1): {'R': -1, 'L': -1, 'W_D':-1, 'W_U':-1 },
    (3,2): {'R': -1, 'L': -1, 'W_D': -1, 'U': -1 },
    (3,3): {'W_R': -1, 'L': -1, 'W_D':-1, 'U': -1 , 'T_R':-1 },
    (3,5): {'R': -1, 'W_L': -1, 'W_D':-1, 'U': -1 , 'T_L':-1 },
    (3,6): {'R': -1, 'L': -1, 'W_D':-1, 'U': -1 },
    (3,7): {'R': -1, 'L': -1, 'W_D':-1, 'W_U': -1 },
    (3,8): {'W_R': -1, 'L': -1, 'W_D':-1, 'U': -1 },
    #(4,0): {'W_R': 0, 'W_L':0, 'W_D':0, 'W_U':0}, # SOL
    (4,1): {'R': -1, 'L':0, 'D':-1, 'W_U':-1},
    (4,2): {'R': -1, 'L': -1, 'D': -1, 'W_U':-1 },
    (4,3): {'W_R': -1, 'L': -1, 'D': -1, 'W_U':-1 },
    (4,5): {'R': -1, 'W_L': -1, 'D': -1, 'W_U':-1 },
    (4,6): {'R': -1, 'L': -1, 'D': -1, 'W_U':-1 },
    (4,7): {'R': -1, 'L':-1, 'W_D':-1, 'W_U':-1},
    (4,8): {'W_R': -1, 'L': -1, 'W_D': -1, 'W_U':-1 },

    (5,0): {'W_R': -1, 'W_L':-1, 'D': -1, 'W_U':-1},
    (5,1): {'R': -1, 'W_L':-1, 'W_D': -1, 'U': -1},
    (5,2): {'R': -1, 'L': -1, 'W_D': -1, 'U': -1 },
    (5,3): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },
    (5,5): {'R': -1, 'W_L': -1, 'D': -1, 'U': -1 },
    (5,6): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },
    (5,7): {'R': -1, 'W_L': -1, 'D': -1, 'W_U': -1 },
    (5,8): {'W_R': -1, 'L': -1, 'D': -1, 'W_U': -1 },

    (6,0): {'R': -1, 'W_L':-1, 'W_D': -1, 'U': -1},
    (6,1): {'R': -1, 'L': -1, 'W_D': -1, 'W_U':-1 },
    (6,2): {'R': -1, 'L': -1, 'D': -1, 'W_U': -1 },
    (6,3): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },
    (6,5): {'R': -1, 'W_L': -1, 'D': -1, 'U':-1 },
    (6,6): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },
    (6,7): {'R': -1, 'W_L': -1, 'W_D': -1, 'U': -1 },
    (6,8): {'W_R': -1, 'L': -1, 'D': -1, 'U': -1 },

    (7,0): {'R': -1, 'W_L':-1, 'W_D':-1, 'W_U':-1 },
    (7,1): {'R': -1, 'L': -1, 'W_D':-1, 'W_U':-1 },
    (7,2): {'R': -1, 'L': -1, 'W_D': -1, 'U': -1 },
    (7,3): {'W_R': -1, 'L': -1, 'W_D':-1, 'U': -1, 'T_R': -1 },
    (7,5): {'R': -1, 'W_L':-1, 'W_D':-1, 'U':-1, 'T_L': -1 },
    (7,6): {'R': -1, 'L': -1, 'W_D':-1, 'U':-1 },
    (7,7): {'R': -1, 'L': -1, 'W_D': -1, 'W_U': -1 },
    (7,8): {'W_R': -1, 'L': -1, 'W_D':-1, 'U': -1},
}

numIterations = 1000

In [13]:
# initialization
V = np.zeros(gridsize)
returns = {(i, j):list() for i in range(gridsize[0]) for j in range(gridsize[1])}
deltas = {(i, j):list() for i in range(gridsize[0]) for j in range(gridsize[1])}

In [14]:
returns

{(0, 0): [],
 (0, 1): [],
 (0, 2): [],
 (0, 3): [],
 (0, 4): [],
 (0, 5): [],
 (0, 6): [],
 (0, 7): [],
 (0, 8): [],
 (1, 0): [],
 (1, 1): [],
 (1, 2): [],
 (1, 3): [],
 (1, 4): [],
 (1, 5): [],
 (1, 6): [],
 (1, 7): [],
 (1, 8): [],
 (2, 0): [],
 (2, 1): [],
 (2, 2): [],
 (2, 3): [],
 (2, 4): [],
 (2, 5): [],
 (2, 6): [],
 (2, 7): [],
 (2, 8): [],
 (3, 0): [],
 (3, 1): [],
 (3, 2): [],
 (3, 3): [],
 (3, 4): [],
 (3, 5): [],
 (3, 6): [],
 (3, 7): [],
 (3, 8): [],
 (4, 0): [],
 (4, 1): [],
 (4, 2): [],
 (4, 3): [],
 (4, 4): [],
 (4, 5): [],
 (4, 6): [],
 (4, 7): [],
 (4, 8): [],
 (5, 0): [],
 (5, 1): [],
 (5, 2): [],
 (5, 3): [],
 (5, 4): [],
 (5, 5): [],
 (5, 6): [],
 (5, 7): [],
 (5, 8): [],
 (6, 0): [],
 (6, 1): [],
 (6, 2): [],
 (6, 3): [],
 (6, 4): [],
 (6, 5): [],
 (6, 6): [],
 (6, 7): [],
 (6, 8): [],
 (7, 0): [],
 (7, 1): [],
 (7, 2): [],
 (7, 3): [],
 (7, 4): [],
 (7, 5): [],
 (7, 6): [],
 (7, 7): [],
 (7, 8): [],
 (8, 0): [],
 (8, 1): [],
 (8, 2): [],
 (8, 3): [],
 (8, 4): [],

In [15]:
acts = list(actions_.keys())

In [16]:
# utils
def generateEpisode():

    initState = random.choice(acts)

    episode = []
    while True:

        if list(initState) in terminationStates:
            return episode

        #return random.choice(list(actions_[initState].keys()))
        action = random.choice(list(actions_[initState].keys()))
        finalState = np.array(initState)+np.array(actions_dictionary[action])

        episode.append([list(initState), actions_dictionary[action], actions_[initState][action], list(finalState)])

        initState = tuple(finalState)

In [17]:
def monte_carlo_first_visit_algorithm(gamma):

    V = np.zeros(gridsize)
    returns = {(i, j):list() for i in range(gridsize[0]) for j in range(gridsize[1])}
    deltas = {(i, j):list() for i in range(gridsize[0]) for j in range(gridsize[1])}

    for it in tqdm(range(numIterations)):
        episode = generateEpisode()
       # print(episode)
        G = 0
        #print(episode)
        for i, step in enumerate(episode[::-1]):
            G = gamma*G + step[2]
            if step[0] not in [x[0] for x in episode[::-1][len(episode)-i:]]:
                idx = (step[0][0], step[0][1])
                returns[idx].append(G)
                newValue = np.average(returns[idx])
                deltas[idx[0], idx[1]].append(np.abs(V[idx[0], idx[1]]-newValue))
                V[idx[0], idx[1]] = newValue


    return deltas, V

### Value function for cooking chef with 10000 iterations, gamma= 0.8 and Monte carlo algorithm

In [19]:
deltas, valueMap = monte_carlo_first_visit_algorithm(0.8)
first_half = pd.DataFrame(valueMap[:4])
second_half = pd.DataFrame(valueMap[4:8])
second_half.index +=4

100%|██████████████████████████████████████| 1000/1000 [00:08<00:00, 117.75it/s]


In [20]:
print(first_half)
print()
print(second_half)

          0         1         2         3    4         5         6         7         8
0 -5.000000 -5.000000 -4.999999 -4.999992  0.0 -4.999993 -4.999997 -4.999996 -4.999999
1 -4.998899 -5.000000 -5.000000 -4.999970  0.0 -4.999994 -4.999995 -4.999469 -4.999677
2 -4.999455 -4.999708 -4.999362 -4.999764  0.0 -4.999998 -4.999993 -4.999543 -4.999659
3 -4.999916 -4.999855 -4.999589 -4.999535  0.0 -4.999990 -4.999984 -4.999954 -4.999847

          0         1         2         3    4         5         6         7         8
4  0.000000 -2.743009 -4.149528 -4.567749  0.0 -4.229287 -3.149804  0.000000 -1.000000
5 -4.995495 -3.971469 -4.468087 -4.702107  0.0 -4.550496 -4.344068 -4.998011 -4.997753
6 -4.992539 -4.981647 -4.960380 -4.895866  0.0 -4.779550 -4.730435 -4.997667 -4.993539
7 -4.994986 -4.991080 -4.978986 -4.947183  0.0 -4.909540 -4.894127 -4.957658 -4.981430


### Value function for cooking chef with 10000 iterations, gamma= 0.9 and Monte carlo algorithm

In [22]:
deltas, valueMap = monte_carlo_first_visit_algorithm(0.9)
first_half = pd.DataFrame(valueMap[:4])
second_half = pd.DataFrame(valueMap[4:8])
second_half.index += 4

100%|██████████████████████████████████████| 1000/1000 [00:09<00:00, 106.89it/s]


In [23]:
print(first_half)
print()
print(second_half)

          0         1         2         3    4         5         6         7         8
0 -9.999910 -9.999749 -9.999221 -9.999477  0.0 -9.999850 -9.999868 -9.999975 -9.999992
1 -9.966693 -9.999550 -9.998592 -9.994750  0.0 -9.999765 -9.999916 -9.978059 -9.991256
2 -9.975593 -9.984765 -9.987968 -9.987378  0.0 -9.999583 -9.999740 -9.990582 -9.990443
3 -9.999556 -9.999096 -9.997138 -9.993686  0.0 -9.996363 -9.997433 -9.997156 -9.996417

          0         1         2         3    4         5         6         7         8
4  0.000000 -4.567307 -7.279727 -8.409243  0.0 -7.262483 -5.292164  0.000000 -1.000000
5 -9.930941 -6.798296 -7.939670 -8.704064  0.0 -8.069324 -7.563649 -9.948296 -9.919345
6 -9.902228 -9.788748 -9.657040 -9.361983  0.0 -8.776021 -8.709291 -9.947174 -9.902599
7 -9.902419 -9.834092 -9.688173 -9.518185  0.0 -9.272147 -9.296197 -9.618801 -9.791249
